## 1-1. 데이터 불러오기

In [1]:
# 데이터 불러오기
import pandas as pd
from pathlib import Path

# === CSV 경로 설정 ===
DATA_PATH = Path("train.csv")
 
# 파일 존재 확인
if not DATA_PATH.exists():
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {DATA_PATH.resolve()}")

# 읽어오기
df = pd.read_csv(DATA_PATH)

# 첫 5행 출력
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 타깃 분포 살피기

In [57]:
print("[타깃 분포]")

# 타깃 컬럼이 데이터에 존재할 때만 실행
if "Survived" in df.columns:
    # 각 클래스(0: 사망, 1: 생존)의 개수 세기
    # dropna=False → 결측치(NaN)가 있으면 그것도 개수에 포함
    print(df["Survived"].value_counts(dropna=False))

    print("\n- 비율:")
    # 각 클래스의 상대적 비율 계산
    # normalize=True → 전체 대비 비율 반환 (합계 = 1)
    # round(3) → 소수점 셋째 자리까지 반올림
    print(df["Survived"].value_counts(normalize=True).round(3))


[타깃 분포]
Survived
0    549
1    342
Name: count, dtype: int64

- 비율:
Survived
0    0.616
1    0.384
Name: proportion, dtype: float64


### 범주형 분포 살피기

In [58]:
print("[주요 범주형 분포: 상위 5개 카테고리]")

# 분석에서 중요한 범주형 변수 몇 개를 골라 분포 확인
for c in ["Pclass", "Sex", "Embarked"]:     
    if c in df.columns:                     # 해당 컬럼이 실제로 존재할 때만 실행
        print(f"\n- {c} 분포:")
        
        # 각 카테고리별 개수를 세어 출력
        # dropna=False → 결측치(NaN)도 포함
        # head(10) → 상위 10개 카테고리까지만 출력 (범주가 많을 경우 일부만 보기)
        print(df[c].value_counts(dropna=False).head(10))


[주요 범주형 분포: 상위 5개 카테고리]

- Pclass 분포:
Pclass
3    491
1    216
2    184
Name: count, dtype: int64

- Sex 분포:
Sex
male      577
female    314
Name: count, dtype: int64

- Embarked 분포:
Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64


### 교차표를 활용한 생존율 분석

In [59]:
print("\n[교차표: 성별별 생존율]")

# "Sex"와 "Survived" 컬럼이 모두 있으면 실행
if set(["Sex","Survived"]).issubset(df.columns):
    # pd.crosstab: 교차표(빈도표) 생성
    # index="Sex", columns="Survived" → 행: 성별, 열: 생존 여부
    # normalize="index" → 각 행을 기준으로 비율 계산 (즉, 성별별 생존율)
    # round(3) → 소수점 셋째 자리까지 반올림
    print(pd.crosstab(df["Sex"], df["Survived"], normalize="index").round(3))


print("\n[교차표: 선실등급별 생존율]")
# "Pclass"와 "Survived" 컬럼이 모두 있으면 실행
if set(["Pclass","Survived"]).issubset(df.columns):
    # 행: Pclass(선실 등급), 열: Survived(생존 여부)
    # normalize="index" → 등급별 생존율 계산
    print(pd.crosstab(df["Pclass"], df["Survived"], normalize="index").round(3))


[교차표: 성별별 생존율]
Survived      0      1
Sex                   
female    0.258  0.742
male      0.811  0.189

[교차표: 선실등급별 생존율]
Survived      0      1
Pclass                
1         0.370  0.630
2         0.527  0.473
3         0.758  0.242


### 그룹 집계를 활용한 평균 생존율 분석

In [ ]:
# "Pclass"와 "Survived" 컬럼이 모두 존재할 때 실행
if set(["Pclass","Survived"]).issubset(df.columns):
    print("\n- Pclass별 평균 생존율")
    # groupby("Pclass"): 선실 등급별로 그룹화
    # ["Survived"].mean(): 각 그룹에서 생존률(평균값) 계산
    # round(3): 소수점 셋째 자리까지 반올림
    print(df.groupby("Pclass")["Survived"].mean().round(3))

# "Sex"와 "Survived" 컬럼이 모두 존재할 때 실행
if set(["Sex","Survived"]).issubset(df.columns):
    print("\n- Sex별 평균 생존율")
    # 성별별 평균 생존율 계산
    print(df.groupby("Sex")["Survived"].mean().round(3))

# "Embarked"와 "Survived" 컬럼이 모두 존재할 때 실행
if set(["Embarked","Survived"]).issubset(df.columns):
    print("\n- Embarked별 평균 생존율")
    # 탑승 항구별 평균 생존율 계산
    print(df.groupby("Embarked")["Survived"].mean().round(3))


- Pclass별 평균 생존율
Pclass
1    0.630
2    0.473
3    0.242
Name: Survived, dtype: float64

- Sex별 평균 생존율
Sex
female    0.742
male      0.189
Name: Survived, dtype: float64

- Embarked별 평균 생존율
Embarked
C    0.554
Q    0.390
S    0.337
Name: Survived, dtype: float64


In [62]:
# 상관관계(수치형만)
print("\n[수치형 상관관계]")

# 분석에 사용할 주요 수치형 컬럼만 골라서 리스트 생성
# 실제 데이터프레임에 존재하는 컬럼만 선택
corr_cols = [c for c in ["Survived","Pclass","Age","SibSp","Parch","Fare"] if c in df.columns]

# 선택한 컬럼들 간의 상관계수(correlation) 계산
# - corr(): 피어슨 상관계수(기본값) 계산
# - numeric_only=True: 숫자형 데이터만 대상으로 계산
corr = df[corr_cols].corr(numeric_only=True)

# 상관계수 소수점 셋째 자리까지 반올림해서 보기 좋게 출력
corr.round(3)


[수치형 상관관계]


,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000,-0.338,-0.077,-0.035,0.082,0.257
Pclass,-0.338,1.000,-0.369,0.083,0.018,-0.549
Age,-0.077,-0.369,1.000,-0.308,-0.189,0.096
SibSp,-0.035,0.083,-0.308,1.000,0.415,0.160
Parch,0.082,0.018,-0.189,0.415,1.000,0.216
Fare,0.257,-0.549,0.096,0.160,0.216,1.000


---

## 1-3. 데이터 분할하기

목표 : 데이터를 "공부용(훈련셋)"과 "시험용(테스트셋)"으로 나눠 모델이 진짜로 잘하는지 공정하게 확인합니다.

In [65]:
from sklearn.model_selection import train_test_split
import pandas as pd

# [1] 정답(타깃)과 입력(피처) 정하기
# - TARGET: 우리가 맞히고 싶은 값(타깃) → 생존 여부
# - FEATURES: 타깃을 예측할 때 참고할 힌트(피처) 목록
TARGET = "Survived"
FEATURES = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# [2] df에서 타깃(y)과 피처(X) 뽑기
# - copy()는 원본 df를 실수로 건드리지 않기 위한 안전장치
y = df[TARGET].copy()
X = df[FEATURES].copy()

# [2-1] (안전) 타깃에 결측값이 있으면 그 행은 학습 불가 → 빼기
#       * 타깃이 비어 있으면 "정답 없는 문제"라서 학습할 수 없음
if y.isna().any():
    keep = y.notna()
    X = X[keep].copy()
    y = y[keep].copy()

# [3] 실제 분할하기
# - test_size=0.2  : 전체의 20%를 시험용(테스트)으로 사용
# - random_state=42: 섞는 시드(고정값). 항상 같은 결과가 나오도록 재현성을 보장
# - stratify=y     : 생존/사망 비율을 train/test에서 비슷하게 유지(불균형 데이터에 중요)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
    shuffle=True  # 기본값 True. (시계열 데이터라면 False로 두고 시간 순서 유지 필요)
)

# [4] 분할 결과 간단 체크
print("[분할 결과]")
print(f"- 전체 행 수: {len(X)}")
print(f"- 훈련셋 행 수: {len(X_train)}")
print(f"- 테스트셋 행 수: {len(X_test)}")

# [5] 타깃(생존/사망) 비율이 비슷한지 확인 → stratify가 잘 됐는지 체크
def ratio(s: pd.Series):
    # value_counts(normalize=True): 비율로 보기, round(3): 소숫점 3자리
    return s.value_counts(normalize=True).sort_index().round(3).to_dict()

print("\n[타깃 비율(생존=1, 사망=0)]")
print("overall:", ratio(y))
print("train  :", ratio(y_train))
print("test   :", ratio(y_test))


[분할 결과]
- 전체 행 수: 891
- 훈련셋 행 수: 712
- 테스트셋 행 수: 179

[타깃 비율(생존=1, 사망=0)]
overall: {0: 0.616, 1: 0.384}
train  : {0: 0.617, 1: 0.383}
test   : {0: 0.615, 1: 0.385}


---

## 1-4 데이터 전처리

In [71]:
# === 중복 처리: ML 최소 버전 ===

# 1) 모든 컬럼이 완전히 같은 행 제거
before = len(df)
df = df.drop_duplicates()
print(f"[drop_duplicates(all-cols)] {before} -> {len(df)}")

# 2) (있으면) 고유키 중복 제거: PassengerId 기준으로 한 건만 남김
if "PassengerId" in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=["PassengerId"], keep="first")
    print(f"[drop_duplicates(PassengerId)] {before} -> {len(df)}")

[drop_duplicates(all-cols)] 891 -> 891
[drop_duplicates(PassengerId)] 891 -> 891
